## GBDT 参数


1) n_estimators: 也就是弱学习器的最大迭代次数，或者说最大的弱学习器的个数。一般来说n_estimators太小，容易欠拟合，n_estimators太大，又容易过拟合，一般选择一个适中的数值。默认是100。在实际调参的过程中，我们常常将n_estimators和下面介绍的参数learning_rate一起考虑。

2) learning_rate: 默认0.1. 数值越小，鲁棒性越好，泛化能力越强，但是需要更多的学习器个数，会增加计算复杂度。通常我们用步长和迭代最大次数一起来决定算法的拟合效果。所以这两个参数n_estimators和learning_rate要一起调参。一般来说，可以从一个小一点的v开始调参，默认是1。

3) subsample: 即我们在原理篇的正则化章节讲到的子采样，取值为(0,1]。注意这里的子采样和随机森林不一样，随机森林使用的是放回抽样，而这里是不放回抽样。如果取值为1，则全部样本都使用，等于没有使用子采样。如果取值小于1，则只有一部分样本会去做GBDT的决策树拟合。选择小于1的比例可以减少方差，即防止过拟合，但是会增加样本拟合的偏差，因此取值不能太低。推荐在[0.5, 0.8]之间，默认是1.0，即不使用子采样。一般来说，取0.8左右，效果比较好。

4) loss:损失函数。分类器默认'deviance',即对数似然损失函数，还有指数损失函数"exponential"
对于回归模型，有均方差"ls", 绝对损失"lad", Huber损失"huber"和分位数损失“quantile”。默认是均方差"ls"。一般来说，如果数据的噪音点不多，用默认的均方差"ls"比较好。如果是噪音点较多，则推荐用抗噪音的损失函数"huber"。而如果我们需要对训练集进行分段预测的时候，则采用“quantile”。

5) random_state 随机数发生器的种子。在调参是，需要将其设为固定的值，但是这样会造成模型过分拟合一组特定的子样本。


##  GBDT调参

首先从步长和分类器个数入手。一般来说，选择一个最小的步长（比如说0.1）来网格搜索最优的迭代次数。  

对决策树最大深度max_depth参数和min_samples_split进行网格搜索，因为max_samples_split还和min_samples_leaf有关，所以不能确定最优值，只能缩小范围
对min_saples_split和min_saples_leaf进行网格搜索调参  

对最大特征数max_features进行网格搜索调参  
对子采样比例sub_samples进行调参

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline

train = pd.read_csv('train_modified.csv')

train = train[:][1:1000]
target = 'Disbursed'
IDCol = 'ID'

D:\common\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\common\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train['Disbursed'].value_counts()

0    988
1     11
Name: Disbursed, dtype: int64

In [3]:
x_columns = [x for x in train.columns if x not in [target,IDCol]]

x = train[x_columns]
y = train[IDCol]


In [ ]:
gbm0 = GradientBoostingClassifier(random_state=10)

gbm0.fit(x,y)
y_pred = gbm0.predict(x)


In [ ]:
help(gbm0)





In [13]:
m=2000
(1-1/m)**m

0.367787452146011

In [2]:
1/10

0.1